# How to use the Molcube membrane builder API


### Part 0: Package installation
---

#### Please use the follwing command to install the package:

In [ ]:
pip install -U http://deployments.molcube.com/api/molcube-0.3.0-py3-none-any.whl

### Part 1. Authentication and API setup

In [15]:
import molcube as mc
import os

# Connect to the MolCube API server.
molcube = mc.API('api.molcube.com', 443)

# Enter the username and password for API authentication.
# (For security, it is recommended to store credentials in a separate file)
# e.g., using: %run keys.py

username = "your_actual_username"
password = "your_actual_password"

%run keys.ipynb # This is my personal keys
molcube.authenticate(username=username, password=password)

print("✅ MolCube API Authentication Successful!")

# Create a directory to store the results.
output_dir = './molcube_results/membrane_system'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

✅ MolCube API Authentication Successful!


### Part 2. Create membrane builder project

In [16]:
membrane_builder = molcube.create_membrane_builder_project()

# Specify the project ID of a previously completed PDBReader job.
# This ID is used to load the membrane protein structure processed by PDBReader.
pdbreader_project_id = "ID of your completed PDBReader job" 
pdbreader_project_id = '02b976aa-028b-48a8-8424-a16d7a380db5' #This is my previous project id   

# Create the Membrane Builder project.
title = "My_Membrane_System"
membrane_builder.create_project(title=title, pdbreader_project_id=pdbreader_project_id)

True

### Part 3. Align protein and pack lipids

In [ ]:
# Automatically align the protein to the membrane plane along the Z-axis.
# (option='ppm' uses the OPM/PPM database to predict the optimal position)
membrane_builder.align_membrane_protein(option='ppm')

# Define the type and ratio of lipids for the membrane, and the padding around the protein.
lipid_composition = {"POPC": 9, "CHL": 1} # e.g., A 9:1 ratio of POPC to Cholesterol
box_margin = {'margin': 20} # 20 Å padding from the protein boundary

# Execute the lipid packing job around the protein based on the defined composition.
membrane_builder.generate_lipid_packing(comp=lipid_composition,box_margin=box_margin)

aligning system (it can take up to five minutes)...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:24<00:00, 68.21s/it]


### Part 4. Generate and Download the simulation system

In [ ]:
membrane_builder.replace_packing()

# Finally, add water and ions to generate the complete system for simulation.
# The mdEngines option allows you to specify file formats for your desired simulation engine.
membrane_builder.generate_system(ions='kcl', mdEngines=['openmm', 'gromacs'])

filename = 'membrane.tar.gz'
output_path = os.path.join(output_dir, file_name)

# Download the final, complete system file.
membrane_builder.download_project(filename=output_path)
print(f"Membrane system files successfully downloaded to '{output_path}'.")